# Predict Employee Burnout

## 1. Business Problem and Understanding

In [ ]:
# Import libraries

import pandas as pd
import numpy as np
import random
np.random.seed(42)
random.seed(42)
import matplotlib.pyplot as plt
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import root_mean_squared_error, ConfusionMatrixDisplay
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, \
GradientBoostingRegressor, VotingRegressor, StackingRegressor
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImPipeline
from xgboost import XGBRegressor
# import warnings
# warnings.filterwarnings("ignore")

## 2. Data Understanding

In [ ]:
# Import data

train = pd.read_csv('Data/train.csv')
test = pd.read_csv('Data/test.csv')
sample_submission = pd.read_csv('Data/sample_submission.csv')

In [ ]:
# Combine train & test

full_df = pd.concat([train,test],axis=0)
full_df.head()

In [ ]:
# Transform column names

full_df.columns = full_df.columns.str.lower()
full_df.columns = full_df.columns.str.replace(' ','_')

It looks like we have some missing values in the last 3 columns of the data set. With so much missing from the `burn_rate` we cannot drop those values, we would ideally impute on them.

In [ ]:
full_df.info()

In [ ]:
full_df.isna().sum()/full_df.shape[0]*100

We can see that all numeri columns are a scale, `burn_rate`, our target is from 0-1. We may want to adjust that to be on a 1-10 scale as the other variables

In [ ]:
full_df.describe()

## 3. Data Exporation

It looks like all of the columns with missingness are mostly normally distributed, so we can confidently impute with the mean.

In [ ]:
plt.hist(full_df['burn_rate']);

print('mean:',np.nanmean(full_df['burn_rate']))
print('median:',np.nanmedian(full_df['burn_rate']))

In [ ]:
plt.hist(full_df['mental_fatigue_score']);

print('mean:',np.nanmean(full_df['mental_fatigue_score']))
print('median:',np.nanmedian(full_df['mental_fatigue_score']))

In [ ]:
plt.hist(full_df['resource_allocation']);

print('mean:',np.nanmean(full_df['resource_allocation']))
print('median:',np.nanmedian(full_df['resource_allocation']))

We also don't have any duplicates so we can drop `employee_id` along with `date_of_joining` since we are working with one year.

In [ ]:
full_df.duplicated(subset=['employee_id']).value_counts()

### Target Variable `burn_rate`

Since we want to conduct a predictive model and better understand which employees are at risk of burnout, we will create a label column defining burnout. Since the metric was on a 0-1 scale, for ease of interpretation, we will increase this scale by 10 and round the number to the nearest whole number.

It looks like we have a slight class imbalance which we should try to address.

In [ ]:
full_df['burn_label'] = full_df['burn_rate'].apply(lambda x: round(x*10) if not np.isnan(x) else x)
full_df['burn_label'].value_counts()

Unfortunately we have to drop the missing values in our target in order to split our data into train/test. This will reduce our dataset quite a bit but hopefully we can get decent results still.

In [ ]:
print('Total size after dropping target nas:',full_df.shape[0])
print('Total missing values of target:',full_df['burn_label'].isna().sum())

y_nas_dropped = full_df.dropna(subset=['burn_label'])

print('Total size after dropping target nas:',y_nas_dropped.shape[0])

## 4. Data Preperation

In [ ]:
# Define X, y and split train/test

X = y_nas_dropped.drop(columns=['date_of_joining','employee_id','burn_label','burn_rate'])
y = y_nas_dropped['burn_label']

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42, stratify=y)
X_train.isna().sum()

In [ ]:
# Define num and cat subpipes
subpipe_num = Pipeline(steps=[('num_impute', SimpleImputer()),
                           ('ss', StandardScaler())])
subpipe_cat = Pipeline(steps=[('ohe', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))])

# Create column transformer
CT = ColumnTransformer(transformers=[('subpipe_num', subpipe_num, [3, 4, 5]),
                                         ('subpipe_cat', subpipe_cat, [0, 1, 2])],
                           remainder='passthrough')
# Initial pipeline
lr_pipe = Pipeline(steps=[('ct', CT),
                            ('model', LinearRegression())])
                            
lr_pipe.fit(X_train, y_train)

In [ ]:
# Baseline score

print('Train Score:', root_mean_squared_error(y_train,lr_pipe.predict(X_train)))

## 5. Linear Regression Model

First, lets see if we can improve our LinearRegression model with testing out our Lasso and Ridge models.

### Lasso

In [ ]:
# Replace in pipeline

lasso_pipe = lr_pipe.set_params(model=Lasso())
lasso_pipe.fit(X_train,y_train)

In [ ]:
# Define your parameter grid
params = {
    'model__alpha':[.0001,.001,.01,1,10,100,1000],
    'model__selection': ['cyclic','random']
    
}

rcv=RandomizedSearchCV(lasso_pipe,param_distributions=params,scoring='neg_root_mean_squared_error',
                 return_train_score=True,cv=5,random_state=42,n_iter=15)

rcv.fit(X_train,y_train)
rcv.best_params_

We actually did worse after cross validation. It looks like our model isn't over/under fitting so that's a bonus. We may want to try a different type of model next.

In [ ]:
# Create quick function to see the scores easier for future models
def rcv_metrics(rcv,model_name,X,y,train_df=None):
    best_estimator = rcv.best_estimator_
    score_dict = {'Val Train Score': -np.mean(rcv.cv_results_['mean_train_score']),
                 'Val Test Score':-np.mean(rcv.cv_results_['mean_test_score']),
                 'Model Name': model_name}
    score_df = pd.DataFrame(score_dict,columns=['Model Name','Val Train Score',
                                                'Val Test Score'], index=range(1))
    if train_df is None:
        pass
    else:
       score_df = pd.concat([train_df,score_df])
       score_df.index = range(len(score_df))
    return score_df, best_estimator
    
train_scores, lasso_best = rcv_metrics(rcv,'Lasso',X_train,y_train)
train_scores

Our score on our test is around the same as on our train, so we know there is no overfitting or underfitting.

In [ ]:
# Function to dislay f1 score and confusion matrix
def test_metrics(model,model_name,X,y,test_df=None):
    score_dict = {'Model Name':model_name,
                  'Train Score': root_mean_squared_error(y_train,model.predict(X_train)),
                  'Test Score': root_mean_squared_error(y, model.predict(X))}
    score_df=pd.DataFrame(score_dict,columns=['Model Name','Train Score','Test Score'],index=range(1))
    if test_df is None:
        pass
    else:
       score_df = pd.concat([test_df,score_df])
       score_df.index = range(len(score_df))
       score_df.sort_values(by='Test Score')
    return score_df
    

# Predict on the test
test_scores = test_metrics(lasso_best,'Lasso',X_test,y_test)
test_scores

### Ridge

Now we can assess the Ridge model in comparison to Lasso to see which type of regularization is best for our problem.

In [ ]:
# Replace in pipeline

ridge_pipe = lr_pipe.set_params(model=Ridge())
ridge_pipe.fit(X_train,y_train)

In [ ]:
# Define your parameter grid
params = {
    'model__alpha':[.0001,.001,.01,1,10,100,1000],
    'model__solver': ['auto','svd','cholesky','lsqr','sag'],
    'model__tol': [1e-5,1e-3,1e-2,1e-1]
    
    
}
rcv=RandomizedSearchCV(ridge_pipe,param_distributions=params,scoring='neg_root_mean_squared_error',
                 return_train_score=True,cv=5,random_state=42,n_iter=15)

rcv.fit(X_train,y_train)
rcv.best_params_

Our Ridge model did so much better than the Lasso on the validation scores and around the same on our test, It is definitely the stronger of the two models

In [ ]:
# Get train and test metrics
train_scores, ridge_best = rcv_metrics(rcv,'Ridge',X_train,y_train,train_scores)
display(train_scores)

test_scores = test_metrics(ridge_best,'Ridge',X_test,y_test,test_scores)
display(test_scores)

### `SMOTE` for Class Imbalance

Since we know we have a class imbalance, lets see how our model does if we incorporate `SMOTE` and balance out our target class, making sure to only resample all classes but the majority.

In [ ]:
# Create new pipe with `SMOTE` and the a logisitic model with best parameters found with RandomizedCV

imb_pipe = ImPipeline(steps=[('ct',CT),
                            ('sm',SMOTE(random_state=42,sampling_strategy='not majority')),
                            ('model', Ridge())])

# Define new parameter grid
params = {
    'model__alpha':[.0001,.001,.01,1,10,100,1000],
    'model__solver': ['auto','svd','cholesky','lsqr','sag'],
    'model__tol': [1e-5,1e-3,1e-2,1e-1]
    
    
}
rcv=RandomizedSearchCV(imb_pipe,param_distributions=params,scoring='neg_root_mean_squared_error',
                 return_train_score=True,cv=5,random_state=42,n_iter=15)

rcv.fit(X_train,y_train)
rcv.best_params_

Looks like `SMOTE` increased our scores substantially! We will leave smote out of the pipeline going forward.

In [ ]:
# Get train and test metrics
train_scores, ridge_sm = rcv_metrics(rcv,'Ridge Sm',X_train,y_train,train_scores)
display(train_scores)

test_scores = test_metrics(ridge_sm,'Ridge Sm',X_test,y_test,test_scores)
display(test_scores)

## 6. DecisionTree Regressors

Let's see if Decision Tree regressors can improve our scores

### Decision Tree

Next we will try a simple Decision Tree to see if the bagging and subspace sampling can get us a more stable score between model and cross validation.

In [ ]:
# Replace in pipeline

dec_pipe = ridge_pipe.set_params(model=DecisionTreeRegressor(random_state=42))
dec_pipe.fit(X_train,y_train)

In [ ]:
# Randomizedsearch CV 

params = {'model__criterion': ['squared_error', 'absolute_error','poisson'],
          'model__max_depth': [None, 10, 20, 30, 40],
          'model__min_samples_split': [2, 5, 10, 20],
          'model__splitter': ['best', 'random'],
          'model__min_samples_leaf': [1, 2, 5, 10]
}

rcv = RandomizedSearchCV(dec_pipe,param_distributions=params,scoring='neg_root_mean_squared_error',
                        return_train_score=True,random_state=42,n_iter=15)

rcv.fit(X_train,y_train)
rcv.best_params_

Looks like our Decision Tree did better than our Ridge but it is underfitting a bit, so hopefully the Random Forest and help address that.

In [ ]:
# Get train and test metrics
train_scores, dec_best = rcv_metrics(rcv,'Decision',X_train,y_train,train_scores)
display(train_scores)

test_scores = test_metrics(dec_best,'Decision',X_test,y_test,test_scores)
display(test_scores)

### Random Forest

Since Random Forests are better at addressing over/under fitting, we will try to see if we can improve our model with this ensemble model.

In [ ]:
# Replace Regressor in pipeline

forest_pipe = ridge_pipe.set_params(model=RandomForestRegressor(random_state=42))
forest_pipe.fit(X_train,y_train)

It takes quite a bit to go through all the options available for some of these ensemble models so we will add a verbose of 3 for these to make sure that the code is running.

In [ ]:
# Randomizedsearch CV 

params = {'model__criterion': ['squared_error', 'poisson'],
          'model__max_depth': [None, 10, 20, 30, 40],
          'model__min_samples_split': [2, 5, 10, 20],
          'model__warm_start': [True,False],
          'model__min_samples_leaf': [1, 2, 5, 10]
}

rcv = RandomizedSearchCV(forest_pipe,param_distributions=params,scoring='neg_root_mean_squared_error',
                        return_train_score=True,random_state=42,n_iter=15,verbose=3)

rcv.fit(X_train,y_train)

In [ ]:
rcv.best_params_

The Random Forest did even better than our Decision Tree on the validation data and test daya. However, it is still underfitting a bit. Lastly we will try some boosting models.

In [ ]:
# Get train and test metrics
train_scores, forest_best = rcv_metrics(rcv,'Forest',X_train,y_train,train_scores)
display(train_scores)

test_scores = test_metrics(forest_best,'Forest',X_test,y_test,test_scores)
display(test_scores)

## 7. Boosting Regressors

Lets see boosting Regressors yeild better results, starting with `GradientRegressor` first.

### GradientBoost

In [ ]:
# Replace in pipeline

gradient_pipe = ridge_pipe.set_params(model=GradientBoostingRegressor(random_state=42))
gradient_pipe.fit(X_train,y_train)

We will set verbose to 3 so that we can confirm the code is running as this CV takes a bit for the boosting algorithms to run when there are this many hyperparameters to tune.

In [ ]:
# Randomizedsearch CV 

params_gb = {'model__max_depth': [3, 5, 10, None],
          'model__learning_rate': [0.01, 0.05, 0.1, 0.2, 0.3],
          'model__n_estimators': [50, 100, 200, 300, 400],
          'model__min_samples_split':  [2, 5, 10, 20],
          'model__max_features': ['sqrt','log2',None],
          'model__subsample': [0.5, 0.75, 1.0],
          'model__min_samples_leaf': [1, 2, 5, 10]
          
}

rcv_gb = RandomizedSearchCV(gradient_pipe,param_distributions=params_gb,scoring='neg_root_mean_squared_error',
                        return_train_score=True,random_state=42,n_iter=18,verbose=3)

rcv_gb.fit(X_train,y_train)

In [ ]:
# Print parameters of best estimate
rcv_gb.best_params_

Our Gradient Boost is underfitting on the validation data but not so much on the train/test data.

In [ ]:
# Get train and test metrics
train_scores, gradient_best = rcv_metrics(rcv_gb,'GradientBoost',X_train,y_train,train_scores)
display(train_scores)

test_scores = test_metrics(gradient_best,'GradientBoost',X_test,y_test,test_scores)
display(test_scores)

### XGBoost

In [ ]:
# Replace Regressor in pipeline

xgboost_pipe = ridge_pipe.set_params(model=XGBRegressor(random_state=42))
xgboost_pipe.fit(X_train,y_train)

In [ ]:
# Randomizedsearch CV 

params_xgb = {'model__learning_rate':  [0.01, 0.05, 0.1, 0.3, 0.5],
          'model__n_estimators': [50, 100, 200, 300, 400],
          'model__min_child_weight':  [3, 5, 7, 9, 11],
          'model__colsample_bytree': [0.5, 0.75, 1.0],
          'model__subsample': [0.5, 0.75, 1.0],
          'model__reg_alpha':  [0, 0.1, 0.5, 1.0],
          'model__reg_lambda':  [0, 0.1, 0.5, 1.0],
          'model__max_depth': [3, 5, 7, 9, 11]
}

rcv_xgb = RandomizedSearchCV(xgboost_pipe,param_distributions=params_xgb,scoring='neg_root_mean_squared_error',
                        return_train_score=True,random_state=42,n_iter=12,verbose=3)

rcv_xgb.fit(X_train,y_train)
rcv_xgb.best_params_

In [ ]:
# Print parameters of the best estimator
rcv_xgb.best_params_

Very similar results as our Gradient Boost, our validation scores are slightly higher so this model might have a bit more variance. This model is also underfitting slighly more on our train/test data, with test scores nearly the same.

In [ ]:
# Get train and test metrics
train_scores, xgb_best = rcv_metrics(rcv_xgb,'XGBoost',X_train,y_train,train_scores)
display(train_scores)

test_scores = test_metrics(xgb_best,'XGBoost',X_test,y_test,test_scores)
display(test_scores)

### AdaBoost

The last boosting model we'll try is AdaBoost with our XGBoost as our base estimator since it had the best score with the least underfitting.

In [ ]:
# Replace Regressor in pipeline

ada_pipe = ridge_pipe.set_params(model=AdaBoostRegressor(random_state=42,estimator=xgb_best['model']))
ada_pipe.fit(X_train,y_train)

In [ ]:
# Randomizedsearch CV 

params_ada = {'model__learning_rate': [0.001, 0.01, 0.1, 0.2, 0.3],
          'model__n_estimators': [50, 100, 200, 300, 400],
          'model__loss': ['linear','square']
}

rcv_ada = RandomizedSearchCV(ada_pipe,param_distributions=params_ada,scoring='neg_root_mean_squared_error',
                        return_train_score=True,random_state=42,n_iter=3,verbose=3)

rcv_ada.fit(X_train,y_train)

In [ ]:
rcv_ada.best_params_

Adaboost is doing slightly better than XGBoost, but not by much.

In [ ]:
# Get train and test metrics
train_scores, ada_best = rcv_metrics(rcv_ada,'AdaBoost',X_train,y_train,train_scores)
display(train_scores)

test_scores = test_metrics(ada_best,'AdaBoost',X_test,y_test,test_scores)
display(test_scores)

## 8. Averaging & Weighted Avereging

Lastly, we will take some of our best models and create combined models to see if we can get the best out of all of them.

### VotingRegressor

In [ ]:
# Create weighted averaging 

estimators = [
    ('ridge',ridge_best),
    ('xgb',xgb_best),
    ('ada',ada_best)
]

voting_pipe = ridge_pipe.set_params(model=VotingRegressor(estimators=estimators, weights=[.2,.4,.4]))

voting_pipe.fit(X_train,y_train)

In [ ]:
# Randomizedsearch CV 

params_voting = {'model__voting': ['hard','soft']
}

rcv_voting = RandomizedSearchCV(voting_pipe,param_distributions=params_w_avg,scoring='neg_root_mean_squared_error',
                        return_train_score=True,random_state=42,n_iter=2,verbose=3)

rcv_voting.fit(X_train,y_train)

In [ ]:
# Print parameters of the beast estimator
rcv_voting.best_params_

Our Voting Regressor did slightly worse than our GradientBoost and XGBoost and is overfitting slightly more.

In [ ]:
# Get train and test metrics
train_scores, voting_best = rcv_metrics(rcv_voting,'Voting',X_train,y_train,train_scores)
display(train_scores)

test_scores = test_metrics(voting_best,'Voting',X_test,y_test,test_scores)
display(test_scores)

### StackingRegressor

In [ ]:
# replace in pipe

estimators = [
    ('ridge',ridge_best),
    ('gb',gradient_best),
    ('xgb',xgboost_best)
]

stacking_pipe = ridge_pipe.set_params(model=StackingRegressor(estimators=estimators))

stacking_pipe.fit(X_train,y_train)

In [ ]:
# Randomizedsearch CV 

params_stacking = {'model__stack_method': ['auto', 'predict_proba', 'decision_function', 'predict'],
                   'model__final_estimator': [xgb_best['model'],ada_best['model']]
}

rcv_stacking = RandomizedSearchCV(stacking_pipe,param_distributions=params_stacking,scoring='neg_root_mean_squared_error',
                        return_train_score=True,random_state=42,n_iter=2,verbose=3)

rcv_stacking.fit(X_train,y_train)

In [ ]:
# Print parameters of the beast estimator
rcv_stacking.best_params_

Looks like Stacking gives us around the same results as Voting! It is also overfitting on the validation data.

In [ ]:
# Get train and test metrics
train_scores, stacking_best = rcv_metrics(rcv_stacking,'Stacking',X_train,y_train,train_scores)
display(train_scores)

test_scores = test_metrics(stacking_best,'Stacking',X_test,y_test,test_scores)
display(test_scores)